## Individual Assignment - ANGGORO Fajar Tri
## General Workflow
### 1) Problem Statement
### 2) Data Preprocessing
### 3) Feature Selection
### 4) Model Development & Benchmarking
### 5) Conclusion

#### 1) Problem Statement

In this case, we're setting up a benchmark experiment: we're going to develop 5 different Machine Learning Models an apply them on a credit Default dataset. The goal of this bencmark experiment is to find the best performing model within a given dataset

#### 2) Data Preprocessing

In [55]:
import pandas as pd
import numpy as np

# read in data
raw = pd.read_csv('./Data/credit_default_train.csv')
raw.head()

# There are some preprocessing steps that needs to be done here: one hot encoding and scaling the data

,cust_id,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,18895,70000.0,1.0,3.0,2.0,34.0,0.0,0.0,0.0,0.0,...,25559.0,26134.0,26715.0,1700.0,1500.0,2000.0,1000.0,1000.0,2000.0,0
1,25102,390000.0,2.0,2.0,2.0,26.0,2.0,2.0,2.0,0.0,...,140387.0,128112.0,115514.0,5000.0,3000.0,5000.0,4548.0,4100.0,3300.0,0
2,28867,60000.0,1.0,1.0,2.0,27.0,0.0,0.0,0.0,0.0,...,26038.0,28607.0,27997.0,1378.0,1406.0,3000.0,3000.0,0.0,923.0,1
3,1842,140000.0,2.0,2.0,1.0,55.0,0.0,0.0,0.0,0.0,...,72391.0,61298.0,62193.0,4200.0,2822.0,2336.0,2588.0,2250.0,2491.0,0
4,3371,50000.0,1.0,1.0,2.0,29.0,2.0,2.0,2.0,0.0,...,1047.0,0.0,0.0,3000.0,0.0,1000.0,0.0,0.0,0.0,1


Let's start preprocessing the categorical variables first

In [56]:
# Before applying the OHE, let's create a flag variable to indicate missing values
raw["fl_missing_edu"] = np.where(raw['EDUCATION'].isnull(), 1, 0)
raw["fl_missing_mar"] = np.where(raw['MARRIAGE'].isnull(), 1, 0)

# treat missing values by categorise them as 'others'
raw['EDUCATION'].fillna(4, inplace=True)
raw['MARRIAGE'].fillna(3, inplace=True)

# Lets do dummy one hot encoding first: Education & Marital Status
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

# make transformer object
transformer = make_column_transformer(
    (OneHotEncoder(), ['EDUCATION', 'MARRIAGE']),
    remainder='passthrough')

# apply fit transform
transformed = transformer.fit_transform(raw)
OHE_df = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())

# let's fix column names 
OHE_df.columns = OHE_df.columns.str.replace('onehotencoder__', '')
OHE_df.columns = OHE_df.columns.str.replace('remainder__', '')

OHE_df.head()


,EDUCATION_0.0,EDUCATION_1.0,EDUCATION_2.0,EDUCATION_3.0,EDUCATION_4.0,EDUCATION_5.0,EDUCATION_6.0,MARRIAGE_0.0,MARRIAGE_1.0,MARRIAGE_2.0,...,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month,fl_missing_edu,fl_missing_mar
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,26715.0,1700.0,1500.0,2000.0,1000.0,1000.0,2000.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,115514.0,5000.0,3000.0,5000.0,4548.0,4100.0,3300.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,27997.0,1378.0,1406.0,3000.0,3000.0,0.0,923.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,62193.0,4200.0,2822.0,2336.0,2588.0,2250.0,2491.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,3000.0,0.0,1000.0,0.0,0.0,0.0,1.0,0.0,0.0


In [57]:
# for Sex column, we first modify the values for females (2 -> 0)
OHE_df["SEX"] = OHE_df["SEX"].replace({2 : 0})

# create flag variable to indicate missing
OHE_df["fl_missing_sex"] = np.where(OHE_df['SEX'].isnull(), 1, 0)
OHE_df["fl_missing_age"] = np.where(OHE_df['AGE'].isnull(), 1, 0)

# impute missing values
OHE_df['SEX'].fillna(1, inplace=True)
OHE_df['AGE'].fillna(OHE_df['AGE'].mean(), inplace=True)

Now let's preprocess the continuous variables

In [58]:
# create flag variable to indicate missing Limit Balance
OHE_df["fl_missing_limbal"] = np.where(OHE_df['LIMIT_BAL'].isnull(), 1, 0)

# For continuous variables, we impute missing values with either 0 or mean, depending the case
OHE_df['LIMIT_BAL'].fillna(OHE_df['LIMIT_BAL'].mean(), inplace=True)

# Pay Status
OHE_df['PAY_0'].fillna(0, inplace=True)
OHE_df['PAY_2'].fillna(0, inplace=True)
OHE_df['PAY_3'].fillna(0, inplace=True)
OHE_df['PAY_4'].fillna(0, inplace=True)
OHE_df['PAY_5'].fillna(0, inplace=True)
OHE_df['PAY_6'].fillna(0, inplace=True)

# Bill Amount
OHE_df['BILL_AMT1'].fillna(OHE_df['BILL_AMT1'].mean(), inplace=True)
OHE_df['BILL_AMT2'].fillna(OHE_df['BILL_AMT2'].mean(), inplace=True)
OHE_df['BILL_AMT3'].fillna(OHE_df['BILL_AMT3'].mean(), inplace=True)
OHE_df['BILL_AMT4'].fillna(OHE_df['BILL_AMT4'].mean(), inplace=True)
OHE_df['BILL_AMT5'].fillna(OHE_df['BILL_AMT5'].mean(), inplace=True)
OHE_df['BILL_AMT6'].fillna(OHE_df['BILL_AMT6'].mean(), inplace=True)

# Pay Amount 
OHE_df['PAY_AMT1'].fillna(OHE_df['PAY_AMT1'].mean(), inplace=True)
OHE_df['PAY_AMT2'].fillna(OHE_df['PAY_AMT2'].mean(), inplace=True)
OHE_df['PAY_AMT3'].fillna(OHE_df['PAY_AMT3'].mean(), inplace=True)
OHE_df['PAY_AMT4'].fillna(OHE_df['PAY_AMT4'].mean(), inplace=True)
OHE_df['PAY_AMT5'].fillna(OHE_df['PAY_AMT5'].mean(), inplace=True)
OHE_df['PAY_AMT6'].fillna(OHE_df['PAY_AMT6'].mean(), inplace=True)

In [59]:
# lets Scale our Dataset so that it becomes less distance sensitive
# we will use robust scaler since it generally could handle outliers
from sklearn.preprocessing import RobustScaler

cols_to_scale = []

# take continuous columns
for col in OHE_df.columns:
    if col.startswith('BILL_A') or col.startswith('PAY_A'):
        cols_to_scale.append(col)
    elif col =="LIMIT_BAL" or col == "AGE":
        cols_to_scale.append(col)
    else:
        continue

# make transformer object
transformer = make_column_transformer(
    (RobustScaler(), cols_to_scale),
    remainder='passthrough')

# apply fit transform
transformed = transformer.fit_transform(OHE_df)
basetable = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())

# let's fix column names 
basetable.columns = basetable.columns.str.replace('robustscaler__', '')
basetable.columns = basetable.columns.str.replace('remainder__', '')
basetable.head()


,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,...,PAY_3,PAY_4,PAY_5,PAY_6,default.payment.next.month,fl_missing_edu,fl_missing_mar,fl_missing_sex,fl_missing_age,fl_missing_limbal
0,-0.368421,0.000000,0.501669,0.027251,0.066354,0.124728,0.162571,0.197891,-0.097561,-0.126585,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.315789,-0.615385,2.616610,2.570802,2.361719,2.373997,2.289261,2.073098,0.707317,0.239268,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.421053,-0.538462,-0.023844,0.014798,0.057089,0.134110,0.214143,0.224964,-0.176098,-0.149512,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,1.615385,1.373617,1.506447,1.136853,1.042080,0.895895,0.947096,0.512195,0.195854,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.473684,-0.384615,-0.241662,-0.189090,-0.286737,-0.355417,-0.382439,-0.366261,0.219512,-0.492439,...,2.0,0.0,0.0,-2.0,1.0,0.0,0.0,0.0,0.0,0.0


In [60]:
# final check
basetable.isna().sum().sum()

0

In [61]:
# fix column ordering: put custid as the first column and target variable as the last

first_column = basetable.pop('cust_id')
last_column = basetable.pop('default.payment.next.month')

# insert column using insert(position,column_name, value) function
basetable.insert(0, 'cust_id', first_column)
basetable.insert(len(basetable.columns), 'default.payment.next.month', last_column)

# export basetable
basetable.to_csv('./Data/basetable.csv', index = False)

#### 3) Feature Selection

For the Feature Selection, We're taking the univariate selection, by comparing each variable with the target variable, calculating the pearson correlation and cross checking the p-value, we can determine which features to select

In [78]:
# read data
base = pd.read_csv('./Data/basetable.csv')
base.head()

,cust_id,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,...,PAY_3,PAY_4,PAY_5,PAY_6,fl_missing_edu,fl_missing_mar,fl_missing_sex,fl_missing_age,fl_missing_limbal,default.payment.next.month
0,18895.0,-0.368421,0.000000,0.501669,0.027251,0.066354,0.124728,0.162571,0.197891,-0.097561,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,25102.0,1.315789,-0.615385,2.616610,2.570802,2.361719,2.373997,2.289261,2.073098,0.707317,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28867.0,-0.421053,-0.538462,-0.023844,0.014798,0.057089,0.134110,0.214143,0.224964,-0.176098,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1842.0,0.000000,1.615385,1.373617,1.506447,1.136853,1.042080,0.895895,0.947096,0.512195,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3371.0,-0.473684,-0.384615,-0.241662,-0.189090,-0.286737,-0.355417,-0.382439,-0.366261,0.219512,...,2.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,1.0


In [64]:
from scipy.stats import pearsonr

selectedFeatures = []
target = "default.payment.next.month"

# check each features and see the corresponding p-value
for column in base.columns:
    if column not in ("cust_id","default.payment.next.month"):
        (pearson,pvalue) = pearsonr(base[column], base[target])
        print(f"{column} - p-value = {pvalue} - selected : {1 if pvalue < 0.001 else 0}")
        if pvalue < 0.001:
            selectedFeatures.append(column)

# remember that we cant drop features that are the result of One hot encoding, as well as flag variables
# from this pearson method, Age is then the only variable that we're going to drop, along with it's flag variable

LIMIT_BAL - p-value = 1.9401679439168946e-94 - selected : 1
AGE - p-value = 0.30074894662134016 - selected : 0
BILL_AMT1 - p-value = 0.004456220048958849 - selected : 0
BILL_AMT2 - p-value = 0.06110200293297901 - selected : 0
BILL_AMT3 - p-value = 0.059179751490243786 - selected : 0
BILL_AMT4 - p-value = 0.15541953422354793 - selected : 0
BILL_AMT5 - p-value = 0.2799639037186581 - selected : 0
BILL_AMT6 - p-value = 0.34569917409094764 - selected : 0
PAY_AMT1 - p-value = 4.495657176789117e-26 - selected : 1
PAY_AMT2 - p-value = 2.415878198431355e-17 - selected : 1
PAY_AMT3 - p-value = 2.5770191922728295e-14 - selected : 1
PAY_AMT4 - p-value = 1.2621316231343818e-19 - selected : 1
PAY_AMT5 - p-value = 2.8641591129990404e-17 - selected : 1
PAY_AMT6 - p-value = 1.6257411504814043e-14 - selected : 1
EDUCATION_0.0 - p-value = 0.13220884812824707 - selected : 0
EDUCATION_1.0 - p-value = 2.391328788140997e-10 - selected : 1
EDUCATION_2.0 - p-value = 6.579218552919837e-06 - selected : 1
EDUCATI

In [79]:
# lets drop age and it's flag variable
base.drop(['AGE', 'fl_missing_age'], axis = 1, inplace = True)

# assign to X and y
X, y = base.iloc[:, 1:-1], base[target]

#### 4) Model Development & Benchmarking

We're now going to develop our model and benchmark them. For each model, we're using gridsearchCV so that it will automatically pick the best hyperparameters

In [103]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt

# Our first model is going to be SGD classifier, a linear model which is fitted through SGD (Stocastic Gradient Descent)
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()

# scoring criteria: AUC & Accuracy
scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score)}

param_SGD = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0], # learning rate
    'max_iter': [200,400,600,800,1000], 
    'loss': ['log'], # logistic regression,
    'penalty': ['l2'] # for linear model
}

# create gs object
sgd_gs = GridSearchCV(
    estimator = sgd,
    param_grid = param_SGD,
    scoring = scoring,
    refit="AUC",
    cv = 5,
    n_jobs = 5,
    verbose = 2
)

# Fitting our GridSearchCV Object
sgd_gs.fit(X, y)

# get results
results = sgd_gs.cv_results_

# print best results & parameter
print(sgd_gs.best_score_)
print(sgd_gs.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
0.7165061192777207
{'alpha': 0.001, 'loss': 'log', 'max_iter': 600, 'penalty': 'l2'}


In [105]:
# put result in a Dataframe
d = {'SGDClassifier': [sgd_gs.best_score_, pd.DataFrame(results)['mean_test_Accuracy'].max()]}
sgd_res = pd.DataFrame(data = d, index=["Best AUC", "Best Accuracy"])
sgd_res

,SGDClassifier
Best AUC,0.716506
Best Accuracy,0.807200


In [106]:
# Our 2nd Model is KNNclassifier
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

# scoring criteria: AUC & Accuracy
scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score)}

param_knn = {
    'n_neighbors': [3,5,7,9,11,13], # number of N
    'weights': ['uniform', 'distance'], # equal weighting or distance weighting
    'p': [1,2] # Manhattan or Euclidean
}

# create gs object
knn_gs = GridSearchCV(
    estimator = knn,
    param_grid = param_knn,
    scoring = scoring,
    refit="AUC",
    cv = 5,
    n_jobs = 5,
    verbose = 2
)

# Fitting our GridSearchCV Object
knn_gs.fit(X, y)

# get results
results = knn_gs.cv_results_

# print best results & parameter
print(knn_gs.best_score_)
print(knn_gs.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
0.7295458573960956
{'n_neighbors': 13, 'p': 2, 'weights': 'uniform'}


In [107]:
# put result in a Dataframe
d = {'KNNClassifier': [knn_gs.best_score_, pd.DataFrame(results)['mean_test_Accuracy'].max()]}
knn_res = pd.DataFrame(data = d, index=["Best AUC", "Best Accuracy"])
knn_res

,KNNClassifier
Best AUC,0.729546
Best Accuracy,0.807450


In [112]:
# Our 3rd Model is Support Vector Classifier (SVM)
from sklearn.svm import SVC

svc = SVC()

# scoring criteria: AUC & Accuracy
scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score)}

param_svc = {
    'C':[1, 2, 3],
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma':['scale', 'auto'],
    'class_weight': ['balanced']
}

# create gs object
svc_gs = GridSearchCV(
    estimator = svc,
    param_grid = param_svc,
    scoring = scoring,
    refit="AUC",
    cv = 5,
    n_jobs = 5,
    verbose = 2
)

# Fitting our GridSearchCV Object
svc_gs.fit(X, y)

# get results
results = svc_gs.cv_results_

# print best results & parameter
print(svc_gs.best_score_)
print(svc_gs.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
0.7492457869524316
{'C': 3, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'rbf'}


In [113]:
# put result in a Dataframe
d = {'SVMClassifier': [svc_gs.best_score_, pd.DataFrame(results)['mean_test_Accuracy'].max()]}
svc_res = pd.DataFrame(data = d, index=["Best AUC", "Best Accuracy"])
svc_res

,SVMClassifier
Best AUC,0.749246
Best Accuracy,0.801550


In [ ]:
# Our 4th Model is Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()

# scoring criteria: AUC & Accuracy
scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score)}

param_gbc = {
    'loss':['deviance', 'exponential'],
    'learning_rate': [1e-3, 1e-2, 1e-1, 1e0],
    'max_depth':[3, 6, 9]
}

# create gs object
gbc_gs = GridSearchCV(
    estimator = gbc,
    param_grid = param_gbc,
    scoring = scoring,
    refit="AUC",
    cv = 5,
    n_jobs = 5,
    verbose = 2
)

# Fitting our GridSearchCV Object
svc_gs.fit(X, y)

# get results
results = svc_gs.cv_results_

# print best results & parameter
print(svc_gs.best_score_)
print(svc_gs.best_params_)